# Z3 Demo

We demonstrate functionality of the [Z3 Theorem Prover](https://github.com/z3prover/z3).
Helpful resources are:
- the extensive Python-API-based [tutorial](http://theory.stanford.edu/~nikolaj/programmingz3.html), which also explains the theoretical background and solving procedures
- the getting-started [guide](https://rise4fun.com/z3/tutorial/guide) using the Z3 input format
- the Python API documentation in [two](https://z3prover.github.io/api/html/namespacez3py.html) [different](https://z3prover.github.io/api/html/z3.html) formats
- the [Wiki](https://github.com/Z3Prover/z3/wiki) on *GitHub*
- the [online prover](https://rise4fun.com/Z3/)

In [1]:
from z3 import *

# Propositional Logic

Besides solving a simple logical problem, the following example also shows that we can easily handle a whole list of variables.

In [2]:
x = Bools(' '.join(['x' + str(i) for i in range(1, 6)]))
solver = Solver()
solver.add(Not(x[0]))
solver.add(And(x[1:4]))
solver.add(Xor(x[2], x[4]))
print('Formula: ' + str(solver))
print('Satisfiable? ' + str(solver.check()))
print('One model: ' + str(solver.model())) # uses the last check()

Formula: [Not(x1), And(x2, x3, x4), Xor(x3, x5)]
Satisfiable? sat
One model: [x3 = True, x2 = True, x1 = False, x4 = True, x5 = False]


Let's have a look at some advanced features of `Z3`.
For example, we can create checkpoints containing a certain set of constraints.
There also is a function to show statistics, but its result is not really well-explained in the official documentation;
[some](https://stackoverflow.com/questions/18491922/interpretation-of-z3-statistics)
[StackOverflow](https://stackoverflow.com/questions/17856574/how-to-interpret-statistics-z3)
[posts](https://stackoverflow.com/questions/6841193/which-statistics-indicate-an-efficient-run-of-z3)
might help.
Finally, we show how to check an assumption together with the contraints of the solver, without adding that new assumption permanently.

In [3]:
print('SMT-LIB2 representation:') # could be loaded from a file
print(solver.to_smt2())
print('Number of checkpoints: ' + str(solver.num_scopes()))
print('Checkpoint...')
solver.push()
print('Number of checkpoints: ' + str(solver.num_scopes()))
print('Add another constraint (solver might attempt incremental solution from last check)...')
solver.add(x[4])
print('Formula: ' + str(solver))
print('Satisfiable? ' + str(solver.check()))
print('Solver statistics:')
print(solver.statistics())
print('Go back to last checkpoint...')
solver.pop()
print('Formula: ' + str(solver))
print('Number of checkpoints: ' + str(solver.num_scopes()))
print('Satisfiable? ' + str(solver.check()))
print('Satisfiable when adding last constraint temporarily? ' + str(solver.check(x[4])))
print('Clear solver...')
solver.reset()
print('Formula: ' + str(solver))

SMT-LIB2 representation:
; benchmark generated from python API
(set-info :status unknown)
(declare-fun x1 () Bool)
(declare-fun x4 () Bool)
(declare-fun x3 () Bool)
(declare-fun x2 () Bool)
(declare-fun x5 () Bool)
(assert
 (not x1))
(assert
 (and x2 x3 x4))
(assert
 (xor x3 x5))
(check-sat)

Number of checkpoints: 0
Checkpoint...
Number of checkpoints: 1
Add another constraint (solver might attempt incremental solution from last check)...
Formula: [Not(x1), And(x2, x3, x4), Xor(x3, x5), x5]
Satisfiable? unsat
Solver statistics:
(:max-memory   3.21
 :memory       3.01
 :mk-bool-var  6
 :num-allocs   833821
 :num-checks   1
 :rlimit-count 244)
Go back to last checkpoint...
Formula: [Not(x1), And(x2, x3, x4), Xor(x3, x5)]
Number of checkpoints: 0
Satisfiable? sat
Satisfiable when adding last constraint temporarily? unsat
Clear solver...
Formula: []


# Arithmetic and Quantifiers

Besides demonstraing basic arithmetic with quantifiers, the following example also shows we case use the `solve()` function instead of a `Solver` object.

In [4]:
x = Int('x')
y = Int('y')
z = Real('z')
solve(x + y == 4, ForAll([z], x * z == z))

[y = 3, x = 1]


# Special Boolean Theories

According to the [tutorial](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-boolean-theories), there are special handlers for certain Boolean cardinality constraints.
We also demonstrate how to access the assignments of the model.

In [5]:
x = Bools(' '.join(['x' + str(i) for i in range(1, 11)]))
solver = Solver()
solver.add(AtLeast(*x[0:5], 2)) # cardinality; here, list needs to be converted to single args
solver.add(AtMost(*x[0:5], 2)) # cardinality
solver.add(Not(x[0]))
solver.add(Xor(x[1], x[2]))
solver.add(PbEq([(x[5], 2), (x[6], 3), (x[7], 4)], 6)) # pseudo-Boolean: 2*x_6 + 3*x_7 + 4*x_8 = 6
solver.add(PbGe([(x[8], 2), (x[9], 2)], 3)) # pseudo-Boolean: 2*x_9 + 2*x_10 >= 3
print('Formula:')
print(str(solver))
print('Satisfiable? ' + str(solver.check()))
print('One model (default conversion to string):')
print(solver.model())
print('Optimal solution (sorted): ' + ', '.join([str(var) + '=' + str(solver.model()[var]) for var in x]))

Formula:
[at-least(x1, x2, x3, x4, x5),
 AtMost((x1, x2, x3, x4, x5), 2),
 Not(x1),
 Xor(x2, x3),
 PbEq(((x6, 2), (x7, 3), (x8, 4)), 6),
 PbGe(((x9, 2), (x10, 2)), 3)]
Satisfiable? sat
One model (default conversion to string):
[x7 = False,
 x4 = True,
 x5 = False,
 x6 = True,
 x8 = True,
 x9 = True,
 x10 = True,
 x2 = False,
 x3 = True,
 x1 = False]
Optimal solution (sorted): x1=False, x2=False, x3=True, x4=True, x5=False, x6=True, x7=False, x8=True, x9=True, x10=True


# Optimization

We demonstrate the optimization functionality by solving a knapsack problem.

In [6]:
import random

selections = Bools(' '.join(['x' + str(i) for i in range(1, 11)]))
random.seed(25)
weights = [random.randint(1, 11) for i in range(1,11)]
print('Weights: ' + str(weights))
utilities = [random.randint(1, 11) for i in range(1,11)]
print('Utilities: ' + str(utilities))
optimizer = Optimize()
objective = optimizer.maximize(Sum([u * s for (u, s) in zip(utilities, selections)]))
optimizer.add(Sum([w * s for (w, s) in zip(weights, selections)]) <= 9)
print('Satisfiable? ' + str(optimizer.check()))
print('Optimal solution:')
print(optimizer.model())
print('Objective value: ' + str(objective.value()))
print('Full optimization problem:')
print(str(optimizer))

Weights: [7, 1, 4, 5, 11, 8, 1, 5, 1, 5]
Utilities: [10, 7, 2, 10, 2, 10, 11, 4, 9, 10]
Satisfiable? sat
Optimal solution:
[x5 = False,
 x1 = False,
 x2 = True,
 x3 = False,
 x4 = False,
 x6 = False,
 x10 = True,
 x7 = True,
 x8 = False,
 x9 = True]
Objective value: 37
Full optimization problem:
(declare-fun x10 () Bool)
(declare-fun x9 () Bool)
(declare-fun x8 () Bool)
(declare-fun x7 () Bool)
(declare-fun x6 () Bool)
(declare-fun x5 () Bool)
(declare-fun x4 () Bool)
(declare-fun x3 () Bool)
(declare-fun x2 () Bool)
(declare-fun x1 () Bool)
(assert (<= (+ (ite x1 7 0)
       (ite x2 1 0)
       (ite x3 4 0)
       (ite x4 5 0)
       (ite x5 11 0)
       (ite x6 8 0)
       (ite x7 1 0)
       (ite x8 5 0)
       (ite x9 1 0)
       (ite x10 5 0))
    9))
(maximize (+ (ite x1 10 0)
   (ite x2 7 0)
   (ite x3 2 0)
   (ite x4 10 0)
   (ite x5 2 0)
   (ite x6 10 0)
   (ite x7 11 0)
   (ite x8 4 0)
   (ite x9 9 0)
   (ite x10 10 0)))
(check-sat)

